Optimizing RFR Models based on Perovskite Compositions
======================================================

**Author:** Panayotis Manganaris



## dependencies



In [1]:
# featurization
import cmcl
from cmcl import Categories
# multi-criterion model evaluation
from yogi.model_selection import summarize_HPO
from yogi.model_selection import pandas_validation_curve as pvc
from yogi.metrics.pandas_scoring import PandasScoreAdaptor as PSA
from yogi.metrics.pandas_scoring import batch_score
# visualization convenience
from spyglass.model_imaging import parityplot

The Intel distribution provides accelerated ml algorithms. Run this
cell before importing the algorithms



In [1]:
from sklearnex import patch_sklearn
patch_sklearn()

In [1]:
# data tools
import sqlite3
import pandas as pd
import numpy as np
# feature engineering
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, Normalizer, StandardScaler
# predictors
from sklearn.ensemble import RandomForestRegressor
## pipeline workflow
from sklearn.pipeline import make_pipeline as mkpipe
from sklearn.model_selection import KFold, GroupKFold
from sklearn.model_selection import learning_curve, validation_curve
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV as gsCV
# model eval
from sklearn.metrics import make_scorer, mean_squared_error, r2_score, explained_variance_score, max_error
#visualization
from sklearn import set_config
import matplotlib.pyplot as plt
import seaborn as sns

## Load Data and Compute Composition Vectors



### Load Curated Subset of Mannodi Data



In [1]:
mannodi = pd.read_csv("./mannodi_data.csv").set_index(["index", "Formula", "sim_cell"])
lookup = pd.read_csv("./constituent_properties.csv").set_index("Formula")

### Compute Compostion Vectors using cmcl



additionally, cmcl offers a convenience function for creating groups
out of the column labels



In [1]:
mc = mannodi.ft.comp() # compute numerical compostion vectors from strings
mc = mc.collect.abx() # convenient site groupings for perovskites data

### generate mix category and assign to index for future imaging



this helps with visualizing the final model performance by tracking
the features through all the model development transformations



In [1]:
mixlog = mc.groupby(level=0, axis=1).count()
mix = mixlog.pipe(Categories.logif, condition=lambda x: x>1, default="pure", catstring="and")
mc = mc.assign(mix=mix).set_index("mix", append=True)
my = my.assign(mix=mix).set_index("mix", append=True)

## Model BG Using Composition Vectors



### Make Composition Pipeline



The model we use relies on a standard chain of data preprocessing
steps. These steps will be encapsulated in a single meta-estimator
called a "pipeline."

This pipeline is constructed to handle the expected form of the 14
dimensional composition space our data covers. The regression is only
performed after an imputer treats N/A constituent quantities as zero
and a normalizer transformer ensures a bounded domain. The regressor
itself is initialized now using only default "hyperparameters."

Hyperparameters control aspects of the regression algorithm's behavior
that are not learned from the data. Later, a hyperparameter
optimization strategy based on exhaustive grid search will chose
parameters that ensure the architecture performs as best it can for
the purposes of inverse design.

The process for determining how suitable a model is for this purpose
is discussed next.



In [1]:
fillna = SimpleImputer(strategy="constant", fill_value=0.0)
cpipe = mkpipe(fillna, Normalizer(), RandomForestRegressor())

### Scoring Scheme



This dataset posits a challenge to successful modeling. We seek to
create a regressor which generalizes well to perovskites of any alloy
character. As seen in the visualizations, the dataset contains mainly
pure, A, B and X-site alloyed perovskites. Some of which are defined
mostly on axes with substantially differing statistical profiles.
Thereby inviting a model to fit to artifacts of the sample
distributions rather than physically meaningful information. Starting
now, great care will be taken to mitigate this.

The composition space is approximated by a set of discrete domains due
the limited nature of the 2x2x2 supercells used to obtain this
data. The pure domain is completely covered by 90 data points. The
alloy domains are combinatorial large in the 14 dimensional component
space under focus, and each has only been sparsely sampled.

Recall, our primary objective is to create a surrogate model of these
domains that can inform future computations needed to exhaustively
explore it.

In order to validate that our model generalizes well, it will be
necessary to score the model's performance with respect to it's
predictions individually over each alloy domain simultaneously with
it's prediction over the union of these domains.



#### prepare subset scoring weights and ordinal group labels



In [1]:
mixweight = pd.get_dummies(mix)
mixcat = pd.Series(OrdinalEncoder().fit_transform(mix.values.reshape(-1, 1)).reshape(-1),
                     index=mc.index).astype(int)

#### Define Scoring Metrics



Nine metrics are used to monitor the fitness of the random forest model.

-   R<sup>2</sup> and Explained Variance scores keep track of the regression's ability to capture the data trend and spread
-   Max Error helps to keep track the largest breakdown in accuracy.
-   Additionally six group-wise RMSE metrics are kept.
    -   total RMSE
    -   A-site RMSE
    -   B-site RMSE
    -   X-site RMSE
    -   XandB-site RMSE
    -   pure RMSE

the PandasScoreAdaptor (PSA) ensures the prediction losses are
weighted correctly when scoring so long as both the targets and the
sample weights passed to the estimators are always pandas objects



In [1]:
site_mse = PSA(mean_squared_error).score
scorings = {'r2': make_scorer(r2_score),
            'ev': make_scorer(explained_variance_score),
            'maxerr': make_scorer(max_error, greater_is_better=False),
            'rmse': make_scorer(mean_squared_error, greater_is_better=False, squared=False),
            'A_rmse': make_scorer(site_mse, greater_is_better=False,
                                  squared=False, sample_weight=mixweight.A),
            'B_rmse': make_scorer(site_mse, greater_is_better=False,
                                  squared=False, sample_weight=mixweight.B),
            'X_rmse': make_scorer(site_mse, greater_is_better=False,
                                  squared=False, sample_weight=mixweight.X),
            'BandX_rmse': make_scorer(site_mse, greater_is_better=False,
                                      squared=False, sample_weight=mixweight.BandX),
            'Pure_rmse': make_scorer(site_mse, greater_is_better=False,
                                     squared=False, sample_weight=mixweight.pure),}

### Make Dedicated Test Train Split



At this point, a dedicated test-train split is made. This split
preserves the proportion of each alloy group in the test and train
partitions, which helps with the final model evaluation.

Beyond this point, all decisions about model optimization will be made
using only the dedicated training partition. The test partition will
be reserved until a final model pipeline is parametrized and
fit. Then, the predictions made on the test partition will either
confirm or deny the model's ability to work outside of the training
domain. The model that ultimately scores best in this respect will be
used to perform inverse design.

The partition is essentially arbitrary. In this case, just enough test
data is held aside to represent at least one sample of each alloy
type in the final tests.



In [1]:
sss = StratifiedShuffleSplit(n_splits=1, train_size=0.8, random_state=None)
train_idx, test_idx = next(sss.split(mc, mixcat)) #stratify split by mix categories
mc_tr, mc_ts = mc.iloc[train_idx], mc.iloc[test_idx]
my_tr, my_ts = my.iloc[train_idx], my.iloc[test_idx]
mixcat_tr, mixcat_ts = mixcat.iloc[train_idx], mixcat.iloc[test_idx]

### Learning Curves &#x2013; Using Deterministically Random Cross Validation



Cross-validation within the training set will be the only way of
checking the generality of any given model instance. So, first, it is
necessary to have some understanding of how much data is needed to
train the model in order for it to have a chance of generalizing.



In [1]:
kf_lc = KFold(n_splits=10, shuffle=True, random_state=111)

Here a splitter is set to produce a 10 cross validation folds on
randomly shuffled data indices. This means that 9 examples of a fit
model will be will be produced for each partition size to be tested.
The 10th fold will likewise be checked 9 times for each corresponding
training. So, effectively at each of the partition sizes tested, the
model is given 9 chances to perform after seeing 90% of the data. This
is generous. If the model fails to generalize under these conditions
at some partition size, it is probably a lost cause.

Notice, the splitter will also perform internal shuffling of the data
indices. (The shuffling occurs before the splitting in each
case). This helps to prevent the model from training on the ordered
groupings in the dataset. For these learning curves, the splitter is
initialized with a deterministic random state. This ensures that each
fold generated at each partition size in the learning curve is not
only comparable to the others in its split, but to those in
neighboring partition sizes as well.

In the future, when performing hyperparameter optimization, the cv
splitters used will use the global random state. This will help to
ensure that the shuffle seen by the myriad fit calls will not be
treated itself as a hyperparameter to be optimized. The eventual final
model should therefore be robust to the order of data.



In [1]:
LC = pvc(learning_curve, cpipe, mc_tr, my_tr.PBE_bg_eV,
         train_sizes=np.linspace(0.1, 1.0, 10), cv=kf_lc, scoring=scorings)
LC = LC.melt(id_vars=["partition"], ignore_index=False).reset_index()

Notice that the error metrics are negated so that, consistently with
the R<sup>2</sup> and ev scores, the greater the number, the better the model
performs.



In [1]:
p = sns.FacetGrid(LC, col="score", hue="partition", col_wrap=3, sharey=False)
p.map(sns.lineplot, "train_sizes", "value")
p.add_legend()

It appears that 3-4 fold cross-validation is sufficient (training with
300/400 or 260/400 points, validating with the compliment)

The learning curve indicates a few things about the bias and variance
of the model in question. The Random Forest's validation scores
continue to rise as the partition size grows. This implies that the
model's generality likely increases with more exposure. Equivalently,
an insufficiently experienced random forest tends to be biased towards
what it has seen.

In support of this, it appears that the variance achievable by the
default random forest comfortably fits the small samples, but is not
as explanatory when more samples challenge it. This is seen in how the
maximum error and the large B-alloy set's rmse worsen with larger
partition sizes.



### Obtain Generality Baseline



The four main alloy classes are similarly represented in the dataset,
So, it is interesting to know if a model trained on only three of them
performs well on the fourth.

To aid in this, the unrefined model is tested on a group-wise
cross-validation k-fold split to get a baseline performance. The final
refined model will be tested in the same way, but using the dedicated
test split. Of all the architectures developed in these notebooks, the
architecture that performs best in this final evaluation will go on to
serve as the genetic algorithm's surrogate

the folds used by this group-aware partition splitter are different
from the number of folds decided on using the learning curves. Here,
each fold consists entirely of members of ONE group. There are four
main groups, so we use four partition. The tiny "BandX" group is mixed
in with the others.



In [1]:
gkf = GroupKFold(n_splits=4)

A function is defined to streamline this test and ensure it cannot
contaminate the estimator to be optimized



In [1]:
def test_generality(estimator, X_tr, y_tr, groups_tr, X_ts, y_ts, groups_ts):
    estimator = clone(estimator) #unfitted, cloned params
    scores = []
    for train_idx, val_idx in gkf.split(X_tr, y_tr, groups=groups_tr):
        tr_val_group_names = groups_tr.iloc[val_idx].index.get_level_values("mix").unique()
        estimator.fit(X_tr.iloc[train_idx], y_tr.iloc[train_idx])
        tr_val_score_series = pd.Series(batch_score(estimator, X_tr.iloc[val_idx], y_tr.iloc[val_idx], **scorings))
        tr_val_score_series.name="_&_".join(tr_val_group_names)
        scores.append(tr_val_score_series)
    tr_val_scores = pd.concat(scores, axis=1).assign(partition="validation")
    scores = []
    for _, val_idx in gkf.split(X_ts, y_ts, groups=groups_ts):
        ts_group_names = groups_ts.iloc[val_idx].index.get_level_values("mix").unique()
        ts_score_series = pd.Series(batch_score(estimator, X_ts.iloc[val_idx], y_ts.iloc[val_idx], **scorings))
        ts_score_series.name="_&_".join(ts_group_names)
        scores.append(ts_score_series)
    ts_scores = pd.concat(scores, axis=1).assign(partition="test")
    group_scores = pd.concat([tr_val_scores, ts_scores]).round(5).drop_duplicates(keep="first")
    return group_scores

In [1]:
test_generality(cpipe, mc_tr, my_tr.PBE_bg_eV, mixcat_tr, mc_ts, my_ts.PBE_bg_eV, mixcat_ts)

These are validation scores.

-   Unsurprisingly, there is substantial error in all the groups in this case.
-   B discrepancy interpretations:
    -   Again, the variance in the B site alloy type is difficult to capture with
        the current parameters.
    -   Also, possibly, The B partition is mostly representative of the others

To elaborate, when the B partition is excluded from training, the model
fails to generalize, but when it is included it generalizes pretty well.

Note: batch<sub>score</sub> defaults to unweighted scoring if the sum of weights
in the given sample equal zero, that is why most of the group-wise rmse
scores are the same.



### Optimize Hyper-parameters for Composition Model



#### define first level of Hyperparameter search



These cells archive the optimization process, running them will only
serve to illustrate the process taken to pick the parameters that
moved on to the following iterations.



##### 1.  construct original Hyper-parameter Space



In [1]:
#"max_depth": [10, 20, 40],
#"min_samples_split": [2, 5, 10]
grid = [
    {'normalizer__norm': ['l1', 'l2', 'max'],
     'randomforestregressor__bootstrap': [True], #build each tree from sample
     'randomforestregressor__ccp_alpha': [0.0, 0.001, 0.002], #cost-complexity pruning
     'randomforestregressor__criterion': ['mse', 'mae'], #['squared_error', 'poisson'], #update sklearn and try these
     'randomforestregressor__max_depth': [25, 20], #investigate dept of constituent trees, limit
     'randomforestregressor__max_features': ['auto', 3, 5], #split after considering
     'randomforestregressor__max_leaf_nodes': [750, 800], #see depth exploration in DT notes
     'randomforestregressor__max_samples': [0.9, 0.6, 0.3], #frac to bag
     'randomforestregressor__min_impurity_decrease': [0.0, 0.3], #0.3 corresponds to the onset of aggressive ccp
     'randomforestregressor__min_impurity_split': [None],
     'randomforestregressor__min_samples_leaf': [1], #just sensible
     'randomforestregressor__min_samples_split': [2, 5], #
     'randomforestregressor__min_weight_fraction_leaf': [0.0], #
     'randomforestregressor__n_estimators': [20, 50, 100],
     'randomforestregressor__n_jobs': [4], #parallelize exec
     'randomforestregressor__oob_score': [True], #use out-of-bag samples to validate (faster)
     'randomforestregressor__random_state': [None], #do not touch
     'randomforestregressor__verbose': [0], 
     'randomforestregressor__warm_start': [False] #make a new forest every time (honest)
     },
    {'normalizer__norm': ['l1', 'l2', 'max'],
     'randomforestregressor__bootstrap': [False], #Build each tree from everything
     'randomforestregressor__ccp_alpha': [0.0, 0.001, 0.002], #cost-complexity pruning
     'randomforestregressor__criterion': ['mse', 'mae'], #['squared_error', 'poisson'],
     'randomforestregressor__max_depth': [25, 20], #investigate dept of constituent trees, limit
     'randomforestregressor__max_features': ['auto', 3, 5], #split after considering
     'randomforestregressor__max_leaf_nodes': [750, 800], #see depth exploration in DT notes
     'randomforestregressor__max_samples': [None], #"bag" everything
     'randomforestregressor__min_impurity_decrease': [0.0, 0.3], #
     'randomforestregressor__min_impurity_split': [None],
     'randomforestregressor__min_samples_leaf': [1], #just sensible
     'randomforestregressor__min_samples_split': [2, 5], #
     'randomforestregressor__min_weight_fraction_leaf': [0.0], #
     'randomforestregressor__n_estimators': [20, 50, 100],
     'randomforestregressor__n_jobs': [4], #parallelize exec
     #oob score not available
     'randomforestregressor__random_state': [None], #do not touch
     'randomforestregressor__verbose': [0], 
     'randomforestregressor__warm_start': [False] #make a new forest every time (honest)
     }
]

##### 1.  Composition model of PBE<sub>BG</sub>



initially, only 3 fold validation is used to save on computation time



In [1]:
cgs = gsCV(estimator=cpipe,
            param_grid=grid,
            cv=3, verbose=1, scoring=scorings, refit="r2", return_train_score=True)
cgs.fit(mc_tr, my_tr.PBE_bg_eV)

Fitting 3 folds for each of 10368 candidates, totalling 31104 fits



##### 1.  Determine next Grid Space to explore



In [1]:
summary, next_grid = summarize_HPO(cgs, grid, topN=10, metric_weights=[1,1,1,1,1,2,1,0,1], strategy="oavg")
summary

In summary:

| |space<sub>0</sub>|space<sub>1</sub>|entropy<sub>0</sub>|entropy<sub>1</sub>|scores<sub>0</sub>|scores<sub>1</sub>|next<sub>0</sub>|next<sub>1</sub>|
|---|---|---|---|---|---|---|---|---|
| normalizer_<sub>norm</sub>|[l1, l2, max]|[l1, l2, max]|1.082051|1.082051|[16.36, 9.28, 4.44]|[16.36, 9.28, 4.44]|[l1]|[l1]|
| bootstrap|[True]|[False]|0.264410|0.365092|[16.57]|[13.51]|[True]|[False]|
| ccp<sub>alpha</sub>|[0.0, 0.001, 0.002]|[0.0, 0.001, 0.002]|0.847720|0.847720|[17.76, 9.72, 2.59]|[17.76, 9.72, 2.59]|[0.0]|[0.0]|
| criterion|[mse, mae]|[mse, mae]|0.677494|0.677494|[12.26, 17.82]|[12.26, 17.82]|[mae]|[mae]|
| max<sub>depth</sub>|[25, 20]|[25, 20]|0.686211|0.686211|[11.56, 18.52]|[11.56, 18.52]|[20]|[20]|
| max<sub>features</sub>|[auto, 3, 5]|[auto, 3, 5]|0.969392|0.969392|[10.38, 5.12, 14.58]|[10.38, 5.12, 14.58]|[auto, 5]|[auto, 5]|
| max<sub>leaf</sub><sub>nodes</sub>|[750, 800]|[750, 800]|0.682295|0.682295|[19.36, 10.72]|[19.36, 10.72]|[750]|[750]|
| max<sub>samples</sub>|[0.9, 0.6, 0.3]|[None]|0.820702|0.365092|[12.81, 1.59, 2.17]|[13.51]|[0.9]|[None]|
| min<sub>impurity</sub><sub>decrease</sub>|[0.0, 0.3]|[0.0, 0.3]|-0.000000|-0.000000|NaN|NaN|[0.0]|[0.0]|
| min<sub>impurity</sub><sub>split</sub>|[None]|[None]|-0.000000|-0.000000|NaN|NaN|[None]|[None]|
| min<sub>samples</sub><sub>leaf</sub>|[1]|[1]|-0.000000|-0.000000|NaN|NaN|[1]|[1]|
| min<sub>samples</sub><sub>split</sub>|[2, 5]|[2, 5]|0.657692|0.657692|[22.72, 7.36]|[22.72, 7.36]|[2]|[2]|
| min<sub>weight</sub><sub>fraction</sub><sub>leaf</sub>|[0.0]|[0.0]|-0.000000|-0.000000|NaN|NaN|[0.0]|[0.0]|
| n<sub>estimators</sub>|[20, 50, 100]|[20, 50, 100]|1.085233|1.085233|[9.72, 10.54, 9.82]|[9.72, 10.54, 9.82]|[50]|[50]|
| n<sub>jobs</sub>|[4]|[4]|-0.000000|-0.000000|NaN|NaN|[4]|[4]|
| oob<sub>score</sub>|[True]|NaN|0.264410|NaN|[16.57]|NaN|[True]|NaN|
| random<sub>state</sub>|[None]|[None]|-0.000000|-0.000000|NaN|NaN|[None]|[None]|
| verbose|[0]|[0]|-0.000000|-0.000000|NaN|NaN|[0]|[0]|
| warm<sub>start</sub>|[False]|[False]|-0.000000|-0.000000|NaN|NaN|[False]|[False]|

With a first eshaustive search and mostly uniform score weights:

-   l1 normalization is much more performant in preparing the domain for random forest
-   mae (more expensive) significantly higher performing &#x2013; suggesting the tree works better when node splits are not biased by extremes.
-   bootstrapping the regressor is marginally more performant
    -   90% sampling is best so far (rfr improves with more exposure, makes sense)
    -   notice: bootstrap sampling appears to rank more frequently in the
        top ten, but no-bootstrap has similar aggregate scores, suggesting
        it appears fewer times but consistently outranks bootstrapping
-   ccp and impurity decrease thresholds do not appear to positively impact performance, however other tree growth limiters do
    -   a node limit of 750 significantly outperformed a relatively unlimited 800
    -   a depth limit of 20 subdevisions significantly outperformed relatively unlimited 25
        (these limit numbers were obtained from depth analysis on a simple DT)
    -   unlimited splitting is significantly more performant that preventing splits below 5 samples/leaf
        (less ability to fit outliers in a split is not helping to improve these scores)
-   So far, more DT estimators is better than less

when B<sub>rmse</sub> score is weighted more heavily:

-   settings that favor model variance are even better
    -   22.72 > 7.36 -> 2 > 5 min samples/leaf compared to 19.9>7.06
-   boot strap sampling increases its lead on no-bootstrap by 1 point.
-   20 estimators actually ranks much higher! less averaging => more bias



#### &#x2013; Iteratively Optimize Hyperparameters



##### 1.  construct subsequent HP space



The dominant setting are kept, but another exhaustive search with
other bounds on max depth and max leaf nodes.

max feature granularity is increased in the successful range

n estimators is a sensitive parameters. it should be checked for
potential overfitting in the performant range



In [1]:
grid = [
    {'normalizer__norm': ['l1'],
     'randomforestregressor__bootstrap': [True],
     'randomforestregressor__ccp_alpha': [0.0],
     'randomforestregressor__criterion': ['mae'],
     'randomforestregressor__max_depth': [15, 20],
     'randomforestregressor__max_features': ['auto', 5, 10],
     'randomforestregressor__max_leaf_nodes': [700, 750],
     'randomforestregressor__max_samples': [0.9], #continuing to give oob validation it's best chance will probably result in bootstraping appearing more often in the top,
     'randomforestregressor__min_impurity_decrease': [0.0],
     'randomforestregressor__min_impurity_split': [None],
     'randomforestregressor__min_samples_leaf': [1],
     'randomforestregressor__min_samples_split': [2],
     'randomforestregressor__min_weight_fraction_leaf': [0.0],
     'randomforestregressor__n_estimators': [20, 50, 100],
     'randomforestregressor__n_jobs': [4],
     'randomforestregressor__oob_score': [True],
     'randomforestregressor__random_state': [None],
     'randomforestregressor__verbose': [0],
     'randomforestregressor__warm_start': [False]},
    {'normalizer__norm': ['l1'],
     'randomforestregressor__bootstrap': [False], #nevertheless, non-bootstrapped model previously scored high despite their decreased frequency in the top 10.
     'randomforestregressor__ccp_alpha': [0.0],
     'randomforestregressor__criterion': ['mae'],
     'randomforestregressor__max_depth': [15, 20],
     'randomforestregressor__max_features': ['auto', 5, 10],
     'randomforestregressor__max_leaf_nodes': [700, 750],
     'randomforestregressor__max_samples': [None],
     'randomforestregressor__min_impurity_decrease': [0.0],
     'randomforestregressor__min_impurity_split': [None],
     'randomforestregressor__min_samples_leaf': [1],
     'randomforestregressor__min_samples_split': [2],
     'randomforestregressor__min_weight_fraction_leaf': [0.0],
     'randomforestregressor__n_estimators': [20, 50, 100],
     'randomforestregressor__n_jobs': [4],
     'randomforestregressor__random_state': [None],
     'randomforestregressor__verbose': [0],
     'randomforestregressor__warm_start': [False]}
]

##### 1.  Composition model of PBE<sub>BG</sub>



In [1]:
cgs = gsCV(estimator=cpipe,
            param_grid=grid,
            cv=3, verbose=1, scoring=scorings, refit="r2", return_train_score=True)
cgs.fit(mc_tr, my_tr.PBE_bg_eV)

Fitting 3 folds for each of 72 candidates, totalling 216 fits



##### 1.  Determine next Grid Space to explore



In [1]:
summary, next_grid = summarize_HPO(cgs, grid, topN=10, metric_weights=[1,1,1,1,1,2,1,0,1], strategy="oavg")
summary

In summary:

| |space<sub>0</sub>|space<sub>1</sub>|entropy<sub>0</sub>|entropy<sub>1</sub>|scores<sub>0</sub>|scores<sub>1</sub>|next<sub>0</sub>|next<sub>1</sub>|
|---|---|---|---|---|---|---|---|---|
| normalizer_<sub>norm</sub>|[l1]|[l1]|-0.000000|-0.000000|NaN|NaN|[l1]|[l1]|
| bootstrap|[True]|[False]|0.210918|0.343871|[29.23]|[7.06]|[True]|[False]|
| ccp<sub>alpha</sub>|[0.0]|[0.0]|-0.000000|-0.000000|NaN|NaN|[0.0]|[0.0]|
| criterion|[mae]|[mae]|-0.000000|-0.000000|NaN|NaN|[mae]|[mae]|
| max<sub>depth</sub>|[15, 20]|[15, 20]|0.675143|0.675143|[16.32, 19.98]|[16.32, 19.98]|[20]|[20]|
| max<sub>features</sub>|[auto, 5, 10]|[auto, 5, 10]|1.097888|1.097888|[13.86, 11.49, 10.95]|[13.86, 11.49, 10.95]|[auto]|[auto]|
| max<sub>leaf</sub><sub>nodes</sub>|[700, 750]|[700, 750]|0.692782|0.692782|[19.07, 17.22]|[19.07, 17.22]|[700]|[700]|
| max<sub>samples</sub>|[0.9]|[None]|0.210918|0.343871|[29.23]|[7.06]|[0.9]|[None]|
| min<sub>impurity</sub><sub>decrease</sub>|[0.0]|[0.0]|-0.000000|-0.000000|NaN|NaN|[0.0]|[0.0]|
| min<sub>impurity</sub><sub>split</sub>|[None]|[None]|-0.000000|-0.000000|NaN|NaN|[None]|[None]|
| min<sub>samples</sub><sub>leaf</sub>|[1]|[1]|-0.000000|-0.000000|NaN|NaN|[1]|[1]|
| min<sub>samples</sub><sub>split</sub>|[2]|[2]|-0.000000|-0.000000|NaN|NaN|[2]|[2]|
| min<sub>weight</sub><sub>fraction</sub><sub>leaf</sub>|[0.0]|[0.0]|-0.000000|-0.000000|NaN|NaN|[0.0]|[0.0]|
| n<sub>estimators</sub>|[20, 50, 100]|[20, 50, 100]|1.076753|1.076753|[7.67, 10.28, 18.34]|[7.67, 10.28, 18.34]|[100]|[100]|
| n<sub>jobs</sub>|[4]|[4]|-0.000000|-0.000000|NaN|NaN|[4]|[4]|
| oob<sub>score</sub>|[True]|NaN|0.210918|NaN|[29.23]|NaN|[True]|NaN|
| random<sub>state</sub>|[None]|[None]|-0.000000|-0.000000|NaN|NaN|[None]|[None]|
| verbose|[0]|[0]|-0.000000|-0.000000|NaN|NaN|[0]|[0]|
| warm<sub>start</sub>|[False]|[False]|-0.000000|-0.000000|NaN|NaN|[False]|[False]|

In an upset, the non-bootstrapped approach losses standing.

-   non-bootstrapped regressions appear less frequently and score much worse when focusing on strongly performing settings
    -   bootstrapping means the model artifically gains more perspective
        by randomly subsamling the training set within each fold per
        estimator, and it uses the excluded "out of bag" samples to
        contribute to the validation score.
-   optimal tree depth is undecided between 15, 20, and 25 (last round) but 20 has won both rounds narrowly
-   unlike before, trees using all features are most performant. 5 was better than 2 by a lot, but 10 and 'auto'=14 better still.
-   the inclination to use more estimators is better ustified now

when weighting B scores, all of this remain true.

-   more estimators => better even more so
-   bootstrapping is even better
-   limits on tree depth are still ambiguous
-   however, max features is less clear cut. B scores probably favor models that look at mostly the 6 B composition vectors.



#### &#x2013; Iteratively Optimize Hyperparameters



##### 1.  construct subsequent HP space



bootsrapping is decidedly better at this point.

There does not appear to be any good reason to restrict the
composition dimensions available to the model



In [1]:
grid = [
    {'normalizer__norm': ['l1'],
     'randomforestregressor__bootstrap': [True],
     'randomforestregressor__ccp_alpha': [0.0],
     'randomforestregressor__criterion': ['mae'],
     'randomforestregressor__max_depth': [15, 20, 25],
     'randomforestregressor__max_features': ['auto'],
     'randomforestregressor__max_leaf_nodes': [650, 700, 750],
     'randomforestregressor__max_samples': [0.9],
     'randomforestregressor__min_impurity_decrease': [0.0],
     'randomforestregressor__min_impurity_split': [None],
     'randomforestregressor__min_samples_leaf': [1],
     'randomforestregressor__min_samples_split': [2],
     'randomforestregressor__min_weight_fraction_leaf': [0.0],
     'randomforestregressor__n_estimators': [50, 75, 100, 125],
     'randomforestregressor__n_jobs': [4],
     'randomforestregressor__oob_score': [True],
     'randomforestregressor__random_state': [None],
     'randomforestregressor__verbose': [0],
     'randomforestregressor__warm_start': [False]},
]

##### 1.  Composition model of PBE<sub>BG</sub>



In [1]:
cgs = gsCV(estimator=cpipe,
            param_grid=grid,
            cv=3, verbose=1, scoring=scorings, refit="r2", return_train_score=True)
cgs.fit(mc_tr, my_tr.PBE_bg_eV)

totalling 108 fits.



##### 1.  Determine next Grid Space to explore



In [1]:
summary, next_grid = summarize_HPO(cgs, grid, topN=10, metric_weights=[1,1,1,1,1,1,1,0,1], strategy="oavg")
summary

In summary:

| |space<sub>0</sub>|entropy<sub>0</sub>|scores<sub>0</sub>|next<sub>0</sub>|
|---|---|---|---|---|
| normalizer_<sub>norm</sub>|[l1]|-0.000000|NaN|[l1]|
| bootstrap|[True]|-0.000000|NaN|[True]|
| ccp<sub>alpha</sub>|[0.0]|-0.000000|NaN|[0.0]|
| criterion|[mae]|-0.000000|NaN|[mae]|
| max<sub>depth</sub>|[15, 20, 25]|1.094351|[12.92, 7.42, 12.23]|[15, 25]|
| max<sub>features</sub>|[auto]|-0.000000|NaN|[auto]|
| max<sub>leaf</sub><sub>nodes</sub>|[650, 700, 750]|1.097582|[14.43, 9.28, 8.86]|[650]|
| max<sub>samples</sub>|[0.9]|-0.000000|NaN|[0.9]|
| min<sub>impurity</sub><sub>decrease</sub>|[0.0]|-0.000000|NaN|[0.0]|
| min<sub>impurity</sub><sub>split</sub>|[None]|-0.000000|NaN|[None]|
| min<sub>samples</sub><sub>leaf</sub>|[1]|-0.000000|NaN|[1]|
| min<sub>samples</sub><sub>split</sub>|[2]|-0.000000|NaN|[2]|
| min<sub>weight</sub><sub>fraction</sub><sub>leaf</sub>|[0.0]|-0.000000|NaN|[0.0]|
| n<sub>estimators</sub>|[50, 75, 100, 125]|1.380655|[10.9, 7.72, 7.78, 6.17]|[50]|
| n<sub>jobs</sub>|[4]|-0.000000|NaN|[4]|
| oob<sub>score</sub>|[True]|-0.000000|NaN|[True]|
| random<sub>state</sub>|[None]|-0.000000|NaN|[None]|
| verbose|[0]|-0.000000|NaN|[0]|
| warm<sub>start</sub>|[False]|-0.000000|NaN|[False]|

-   substantially limiting nodes appears to be good
-   either strong or lax limits on depth are favored.
-   suddenly less estimators are prefered.
    -   n<sub>estimators</sub> is a dominant setting. It is also more conceptually clear.

So, n<sub>estimators</sub> should be individually optimized at this point, in
order to decide the tree growth limits



#### Perform a sensitivity analysis in this optimal subspace



plot a validation curve for n-estimators when considering unlimited trees parameters.

If limiting trees on top of an "optimal" ensemble size improves the model, good.

Adjusting the ensemble size to fit the tree optimizations is not as
much a sensible use of the RFR architecture's strengths.



##### 1.  construct subsequent HP space



In [1]:
grid = [
    {'normalizer__norm': ['l1'],
     'randomforestregressor__bootstrap': [True],
     'randomforestregressor__ccp_alpha': [0.0],
     'randomforestregressor__criterion': ['mae'],
     'randomforestregressor__max_depth': [None], #unlimited
     'randomforestregressor__max_features': ['auto'],
     'randomforestregressor__max_leaf_nodes': [None], #unlimited
     'randomforestregressor__max_samples': [0.9],
     'randomforestregressor__min_impurity_decrease': [0.0],
     'randomforestregressor__min_impurity_split': [None],
     'randomforestregressor__min_samples_leaf': [1],
     'randomforestregressor__min_samples_split': [2],
     'randomforestregressor__min_weight_fraction_leaf': [0.0],
     'randomforestregressor__n_estimators': [100],
     'randomforestregressor__n_jobs': [4],
     'randomforestregressor__oob_score': [True],
     'randomforestregressor__random_state': [None],
     'randomforestregressor__verbose': [0],
     'randomforestregressor__warm_start': [False]},
]

In [1]:
#set the parameters settled in the grid. n estimators will be handled by the validation curve function in the next step
cpipe = cpipe.set_params(**{k:v[0] for k,v in grid[0].items()})

##### n<sub>estimators</sub> validation scan



using the 4 fold cross validation established by the LC analysis



In [1]:
VC = pvc(validation_curve, cpipe, mc_tr, my_tr.PBE_bg_eV,
         param_name='randomforestregressor__n_estimators', param_range=np.linspace(50, 150, 15).astype(int), cv=4, scoring=scorings)
VC = VC.melt(id_vars=["partition"], ignore_index=False).reset_index()

In [1]:
p = sns.FacetGrid(VC, col="score", hue="partition", col_wrap=3, sharey=False)
p.map(sns.lineplot, 'randomforestregressor__n_estimators', "value")
p.add_legend()

unfortunately, it seems adjustments to ensemble size have no
appreciable effect at these settings. this is a good indication that
this data does not suit the RFR architecture.

This visualization also helps to clarify that the current model space
has reasonable RMSE, but nevertheless consistently permits at least one
large outlier.

There's not much point in continuing. with more minor optimizations if
this parameter is ineffective. There is also huge variability within
each fold. The band here is a 95% confidence interval. Trying to pick
a estimator that narrows this band is not helpful at this point
because sometimes the performance is actually good with the wide band,
whereas the mean performance is catastrophically bad everywhere

we're likely to see better with other models. let's wrap up and move on



### Best Model



#### Parametrize



In [1]:
grid = [
    {'normalizer__norm': ['l1'],
     'randomforestregressor__bootstrap': [True],
     'randomforestregressor__ccp_alpha': [0.0],
     'randomforestregressor__criterion': ['mae'],
     'randomforestregressor__max_depth': [None], #unlimited
     'randomforestregressor__max_features': ['auto'],
     'randomforestregressor__max_leaf_nodes': [None], #unlimited
     'randomforestregressor__max_samples': [0.9],
     'randomforestregressor__min_impurity_decrease': [0.0],
     'randomforestregressor__min_impurity_split': [None],
     'randomforestregressor__min_samples_leaf': [1],
     'randomforestregressor__min_samples_split': [2],
     'randomforestregressor__min_weight_fraction_leaf': [0.0],
     'randomforestregressor__n_estimators': [100], #better performance by somome metrics, about as good as it gets
     'randomforestregressor__n_jobs': [4],
     'randomforestregressor__oob_score': [True],
     'randomforestregressor__random_state': [None],
     'randomforestregressor__verbose': [0],
     'randomforestregressor__warm_start': [False]},
]

#### Train Final Estimator



In [1]:
cpipe = cpipe.set_params(**{k:v[0] for k,v in grid[0].items()})
cpipe.fit(mc_tr, my_tr.PBE_bg_eV)

#### evaluate



In [1]:
p, data = parityplot(cpipe, mc_ts, my_ts.PBE_bg_eV.to_frame(), aspect=1.0, hue="mix")
p.figure.show()

In [1]:
p.savefig("./ParityPlots/rfr_c_bg.png", transparent=True)

In [1]:
pd.Series(batch_score(cpipe, mc_tr, my_tr.PBE_bg_eV, **scorings)).to_frame()

Luckily, the RFR designed here does fit the test data well. Even
despite the possible shortcomings in the architecture.



### Obtain Generality Measure



In [1]:
test_generality(cpipe, mc_tr, my_tr.PBE_bg_eV, mixcat_tr, mc_ts, my_ts.PBE_bg_eV, mixcat_ts)

The post-optimization RFR generalizes across all alloy types many
times better than the default parameters.



## Exporting/Importing Trained Model



### save the model for distribution



Never load joblib/pickle files that you do not trust, they can execute
arbitrary code on your computer.



In [1]:
joblib.dump(cpipe, "./Models/rfr_c_opt.joblib")

### load if needed



In [1]:
cpipe = joblib.load("./Models/rfr_c_opt.joblib")

## Compute Site-Averaged Properties Vectors



In [1]:
mp = mc.ft.derive_from(lookup, "element", "Formula")

## Model BG Using Site-Averaged Properties



### Make Properties Pipeline



In [1]:
ppipe = mkpipe(StandardScaler(), RandomForestRegressor())

## comp+prop model opt

